# Homework 8
## Group C21
* Daniel Noriega
* Fernando Urbano
* Xianbin Xu

## 1. Conceptual issues for LTCM

### 1.1 Describe LTCM’s investment strategy with regard to the following aspects: 

* **Securities traded**

Interest-rate swaps, fixes-rate residential mortgages, government bonds and futures on those bonds, put and call options on broad stock market indexes, as well as individual stocks, among others. LTCM deployed its strategies principally in the UK, French, German and Japanese markets.

* **Trading frequency**

LTCM focused on medium- to long-term opportunities. Positions may have needed to be rebalanced more frequently, but the horizon of their strategies commonly spanned several months, or years. 


* **Skewness (Do they seek many small wins or a few big hits?)**

LTCM focuses on small (but several) wins over long horizons.

* **Forecasting (What is behind their selection of trades?)**

Their strategies, LTCM belived, arose as a result of dislocactions in financial markets caused by insitutional demands, which would take an uncertain amount of time to be corrected.

### 1.2. What are LTCM’s biggest advantages over its competitors?

LTCM could (1) finance its trades very efficiently. In addition, (2) LTCM considered they had paritular expertise in the development of trading strategies using sophisticated analytical models. 

### 1.3. The case discusses four types of funding risk facing LTCM. The case discusses specific ways in which LTCM manages each of these risks.

* **collateral haircuts**

LTCM uses total-return swaps to finance some of its positions with zero haircuts. Its costs of carrying a position was limited to the spread it paid on the swaps. 

* **repo maturity**

LTCM uses longer term maturity repos (6-12 months) as opposed to overnight and short-term repos to prevent difficulties when rolling over repo financing. It also monitored LTCM's repo financing maturity's structure constantly to ensure healthiness.

* **equity redemption**

Establishing withdrawal schemes that prevented investor from withdrawing large amounts/amounts that could endanger the company's financing in short horizons. LTCM staggered the maturiries of the Fund's equity capital.

* **loan access**

LTCM obtained $230 million of unsecured term loans. In addition, the firm negotiated a $700 million revolving line of credit. LTCM insisted that the agreement contained no material adverse change clauses or any other covenant subject to discretionary interpretation by the lenders.

### 1.4. LTCM is largely in the business of selling liquidity and volatility. Describe how LTCM accounts for liquidity risk in their quantitative measurements.

LTCM estimated worst case scenario haircuts under severe economic stress situations. Then, the fund was careful to structure its financing so taht the fund would not be forced to liquidate positions swiftly due to financial market disruptions only.

### 1.5. Is leverage risk currently a concern for LTCM?

By the end of 1997, the fund was leveraged at a level equivalent to the lowest value of the historical range for the fund since it had reached global scale in 1995. Consequenlty, no. Although managed, it was not a major concern for LTCM. 

### 1.6. Many strategies of LTCM rely on converging spreads. LTCM feels that these are almost win/win situations because of the fact that if the spread converges, they make money. If it diverges, the trade becomes even more attractive, as convergence is still expected at a future date. What is the risk in these convergence trades?

Even if convergence were guaranteed, some trades exposed LTCM to operating costs that could erode the firm's viability. If the carry costs of enough trades became negative and large, this could strain the funds bottom line and overall operability. 

#### <i>Importing modules, required functions and data:</i>


In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy.stats import skew, kurtosis

def calculate_statistics(df, annualize_factor=12, VaR=0.05, CVaR=0.05, dropdown=False):
    '''
    Calculates the mean, volatility, sharpe, skewness, kurtosis, VaR, CVaR and dropdown stats of a dataframe
    Returns a dataframe with values for each asset
    '''
    res={}
    res_i={}
    for i in df.columns:
        if df[i].dtype=='<M8[ns]':
            Dates=df[i]
        else:
            res_i.update({'mean':np.mean(df[i])*annualize_factor})
            res_i.update({'volatility':np.std(df[i])*(annualize_factor**(1/2))})
            res_i.update({'sharpe':res_i['mean']/res_i['volatility']})
            res_i.update({'skewness':skew(df[i])})
            res_i.update({'kurtosis':kurtosis(df[i])})
            res_i.update({'VaR':df[i].quantile(VaR)})
            res_i.update({'CVaR':df[i][df[i]<df[i].quantile(CVaR)].mean()})
            if dropdown:
                res_i.update({'Max_Drawdown':maxDrawD(Dates,df[i])})
            res.update({i:res_i})
            res_i={}
    return pd.DataFrame(res)

def maxDrawD(dates, values):
    """
    Calculates the maximum drawdown and the dates of the max/min/recovery within the max drawdown period.
    
    Parameters:
    dates (array): An array of dates
    values (array): An array of values
    
    Returns:
    tuple: A tuple containing the maximum drawdown, the start date of the max drawdown period, 
    the end date of the max drawdown period, the date of the minimum value, and the date of recovery.
    """
    df = pd.DataFrame({'Date': dates, 'Value': values})
    df.dropna(inplace=True)
    dates = df['Date'].values
    values = df['Value'].values

    max_value = values[0]
    max_date = dates[0]
    max_drawdown = 0
    max_drawdown_start_date = dates[0]
    max_drawdown_min_date = dates[0]
    max_recovery_date = dates[0]
    for i in range(1, len(values)):
        if values[i] > max_value:
            max_value = values[i]
            max_date = dates[i]
            max_recovery_date = dates[i]
        else:
            drawdown = (max_value - values[i]) / max_value
            if drawdown > max_drawdown:
                max_drawdown = drawdown
                max_drawdown_start_date = max_date
                max_drawdown_min_date = dates[i]
                try:
                    max_recovery_date = df['Date'].loc[df[(df['Date']>max_date) & (df['Value']>=max_value)].index[0]]
                except:
                    max_recovery_date = np.nan

    res={}
    res.update({'max_drawdown':max_drawdown})
    res.update({'max_drawdown_start_date':max_drawdown_start_date})
    res.update({'max_drawdown_min_date':max_drawdown_min_date})
    res.update({'max_recovery_date':max_recovery_date})
    
    return res

pd.set_option('display.float_format', '{:.5f}'.format)

In [2]:
dfltcm = pd.read_excel('../data/ltcm_exhibits_data.xlsx', sheet_name='Exhibit 2', skiprows=2)
dfltcm.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
dfltcm.dropna(inplace=True)
dfltcm['Date']=pd.to_datetime(dfltcm['Date'])
dfltcm.reset_index(drop=True, inplace=True)

dfrf=pd.read_excel('../data/gmo_analysis_data.xlsx', sheet_name='returns (total)')
dfrf.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
tdfrf=pd.read_excel('../data/gmo_analysis_data.xlsx', sheet_name='risk-free rate')
tdfrf.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
dfrf=dfrf.merge(tdfrf, on='Date', how='left')
del(tdfrf)
dfrf.drop(['GMWAX'], axis=1, inplace=True)
dfrf=dfrf[(dfrf['Date']>='1994-02') & (dfrf['Date']<=dfltcm['Date'].max())]
dfrf.reset_index(drop=True, inplace=True)

dfct_rf=pd.read_excel('../data/fx_carry_data.xlsx',sheet_name='risk-free rates')
dfct_fx=pd.read_excel('../data/fx_carry_data.xlsx',sheet_name='fx rates')
dfct_fx.rename(columns={'USUK':'GBP1M','USEU':'EUR1M','USSZ':'CHF1M','USJP':'JPY1M'}, inplace=True)


## 2. LTCM Risk Decomposition

### 2.1. Summary Stats

<i> (a), (b) </i>

In [3]:
print('LTCM Statistics')
calculate_statistics(dfltcm)[['Gross Monthly Performancea','Net Monthly Performanceb']]

LTCM Statistics


,Gross Monthly Performancea,Net Monthly Performanceb
mean,0.29389,0.20717
volatility,0.13506,0.11084
sharpe,2.17595,1.86903
skewness,-0.28797,-0.79454
kurtosis,1.31405,2.52746
VaR,-0.02640,-0.02240
CVaR,-0.06900,-0.06467


In [4]:
calculate_statistics(dfrf)

,SPY,US3M
mean,0.22209,0.05150
volatility,0.11274,0.00147
sharpe,1.96985,35.06494
skewness,-0.42443,-1.35914
kurtosis,-0.50494,2.57217
VaR,-0.04280,0.00345
CVaR,-0.04696,0.00306


<i> (c) </i> The difference between gross and net performances is substantial (~1/3). The gross performance is better than SPY's for the same period, but on a net basis they are very similar (SPY actually seems to outperform marginally (better sharpe, lower kurtosis, better CVaR; although worse VaR).

### 2.2. Estimate regression

$$
\tilde{r}_t^{LTCM} = \alpha + \beta^m\tilde{r}_t^m + \epsilon_t
$$

<i> (a) </i>

In [5]:
model=LinearRegression()
model.fit(np.array(dfrf['SPY']).reshape(-1,1),np.array(dfltcm['Net Monthly Performanceb']).reshape(-1,1))
alpha=model.intercept_[0]*12
beta=model.coef_[0][0]
r2=r2_score(np.array(dfltcm['Net Monthly Performanceb']).reshape(-1,1),model.predict(np.array(dfrf['SPY']).reshape(-1,1)))
print('\u03B1',':  ', round(alpha,5))
print('\u03B2',':  ', round(beta, 5))
print('R-squared: ', round(r2,5))

α :   0.19595
β :   0.05051
R-squared:  0.00264


<i> (b) </i> From this regression, it does not seem like LTCM is closet indexing. &alpha; is quite substantial.

<i> (c) </i> Yes. Again, &alpha; is large and positive.

### 2.3. Checking non-linear market exposure.

$$
\tilde{r}_t^{LTCM} = \alpha + \beta_1\tilde{r}_t^m + \beta_2(\tilde{r}_t^m)^2 + \epsilon_t
$$

<i> (a) </i>

In [6]:
model=LinearRegression()
model.fit(pd.DataFrame([dfrf['SPY'],dfrf['SPY'].apply(lambda x: x**2)]).transpose(),np.array(dfltcm['Net Monthly Performanceb']).reshape(-1,1))
alpha=model.intercept_[0]*12
beta1=model.coef_[0][0]
beta2=model.coef_[0][1]
r2=r2_score(np.array(dfltcm['Net Monthly Performanceb']).reshape(-1,1),model.predict(pd.DataFrame([dfrf['SPY'],dfrf['SPY'].apply(lambda x: x**2)]).transpose()))
print('\u03B1',':  ', round(alpha, 5))
print('\u03B2','1:  ', round(beta1, 5))
print('\u03B2','2:  ', round(beta2, 5))
print('R-squared: ', round(r2, 5))

α :   0.25304
β 1:   0.16403
β 2:   -4.8927
R-squared:  0.03714


<i> (b) </i> The quadratic market factor increases the variability explained by a factor greater than 10. Nonetheless, it remains quite small overall, and &alpha; remains large.

<i> (c) </i> Given the large and negative &beta;2, LTCM seems to behave as if it were short market options. However, given that &beta;2 accompanies returns-squared, which are 'small' numbers, the magnitude may not be as large as initially perceived.

<i> (d) </i> Based on the regression above, LTCM seems to be negatively exposed to market volatility. &beta;2 is negative, while options increase in value as volatility increases.

### 2.4. Trying to pinpoint the nature of LTCM's nonlinear exposure.

$$
\tilde{r}_t^{LTCM} = \alpha + \beta\tilde{r}_t^m + \beta_u max(\tilde{r}_t^m - k_1, 0) + \beta_d max(k_2 - \tilde{r}_t^m, 0) +\epsilon_t
$$

<i> (a) </i>

In [7]:
model=LinearRegression()
X=pd.DataFrame(
    [
        dfrf['SPY'],
        (dfrf['SPY']-0.03).apply(lambda x: max(x, 0)),
        (-0.03-dfrf['SPY']).apply(lambda x: max(x, 0))
    ]
)
model.fit(X.transpose(),np.array(dfltcm['Net Monthly Performanceb']).reshape(-1,1))
alpha=model.intercept_[0]*12
beta1=model.coef_[0][0]
beta2=model.coef_[0][1]
beta3=model.coef_[0][2]
r2=r2_score(np.array(dfltcm['Net Monthly Performanceb']).reshape(-1,1),model.predict(X.transpose()))
print('\u03B1',':  ', round(alpha, 5))
print('\u03B2',':  ', round(beta1, 5))
print('\u03B2','u:  ', round(beta2, 5))
print('\u03B2','d:  ', round(beta3, 5))
print('R-squared: ', round(r2, 5))

α :   0.18458
β :   0.45268
β u:   -1.08048
β d:   1.14791
R-squared:  0.06696


<i> (b) </i> LTCM seems to be short the call-like factor (&beta;u), and long the put-like factor (&beta;d).

<i> (c) </i> The absolute value of the put-like factor is higher. The put-like factor seems to affect LTCM's performance more.

<i> (d) </i> Based on this regression, it seems like this exposure stems from shorting the market's upside and being long the market's downside. The fact that &beta;u and &beta;d are of opposite signs strengthens the argument.

## 3. The FX Carry Trade

### 3.1. The Static Carry Trade

<i> (a), (b), (c) </i>

In [8]:
for i in dfct_rf.columns[1:]:
    dfct_rf[i]=np.log(dfct_rf[i]+1)
for i in dfct_fx.columns[1:]:
    dfct_fx[i]=np.log(dfct_fx[i])

In [9]:
elr=pd.DataFrame()
for i in dfct_fx.columns[1:]:
    elr[i]=dfct_fx[i]-dfct_fx[i].shift(1)-dfct_rf['USD1M']+dfct_rf[i]
elr.dropna(inplace=True)
calculate_statistics(elr)

,GBP1M,EUR1M,CHF1M,JPY1M
mean,-0.00354,-0.00431,0.00442,-0.01725
volatility,0.08620,0.09460,0.09863,0.09141
sharpe,-0.04102,-0.04555,0.04478,-0.18876
skewness,-0.30973,-0.12406,0.20106,-0.17586
kurtosis,1.31516,1.13954,2.11007,0.69806
VaR,-0.03961,-0.04487,-0.04189,-0.04918
CVaR,-0.05701,-0.06218,-0.05668,-0.06249


All currencies seem to have similar volatility figures and very small means. The Swiss Franc is the only one that appreciated relative to the dollar over the period analyzed. The dollar strengthened relative to all other currencies.

### 3.2. Implications for UIP

<i> (a) </i> According to UIP we should observe a mean value of 0, which is almost what we obtained. Nevertheless, UIP is a theory about expectations, so these values certainly do not disprove it.

<i> (b) </i> The Swiss Franc was the only currency resulting in a positive sharpe value over the term assessed.

<i> (c) </i> Long positions in all other currencies would have resulted in a negative excess return over the sample (i.e. GBP, EUR and JPY).


### 3.3. Predicting FX

Test wheter interest-rate differentials can predict growth in the foreign-exchange rate:

$$
s_{t+1}^i - s_t^i = \alpha^i + \beta^i(r_{t,t+1}^{f,\$} - r_{t,t+1}^{f,i}) +\epsilon_{t+1}^i
$$

<i> (a) </i>

In [10]:
res=pd.DataFrame()
for i in dfct_fx.columns[1:]:
    model=LinearRegression()
    X=(dfct_rf['USD1M']-dfct_rf[i]).iloc[1:]
    Y=(dfct_fx[i]-dfct_fx[i].shift(1)).dropna()
    model.fit(np.array(X).reshape(-1,1),np.array(Y).reshape(-1,1))
    alpha=model.intercept_[0]
    beta=model.coef_[0][0]
    r2=r2_score(np.array(Y).reshape(-1,1),model.predict(np.array(X).reshape(-1,1)))
    res[i]=[alpha,beta,r2]
res.rename({0:'alpha',1:'beta',2:'r2'}, axis=0, inplace=True)
res

,GBP1M,EUR1M,CHF1M,JPY1M
alpha,-0.00063,0.00074,0.00413,-0.00010
beta,0.11067,-1.63381,-2.06667,0.10538
r2,0.00002,0.00440,0.00616,0.00004


<i> (b) </i> Supposing the foreign risk-free rate increases relative to the US rate:

i. We would predict a relative strengthening of the USD for currencies with a positive beta from the regression above. Consequently, we would predict relative strengthenings of the USD for GBP and JPY.

ii. Conversely (negative beta), we would predict relative weakenings of the USD for EUR and CHF.

iii. This prediction would be strongest in the case of CHF (highest R-squared figure).

### 3.3. The Dynamic Carry Trade

Using:

$$
\mathbb{E}[\tilde{r}_{t+1}^i] = \alpha + (\beta-1)(r_{t,t+1}^{f,\$} - r_{t,t+1}^{f,i})
$$

<i> (a) </i>

In [11]:
tres_E=pd.DataFrame()
for i in res.columns:
    tres_E[i]=res.loc['alpha', i]+(res.loc['beta', i]-1)*(dfct_rf['USD1M']-dfct_rf[i])

print('Fraction of time expected to be positive:  ')
for i in tres_E.columns:
    print(i, round(len(tres_E[tres_E[i]>0])*100/len(tres_E),2),'%')

Fraction of time expected to be positive:  
GBP1M 25.55 %
EUR1M 51.09 %
CHF1M 63.14 %
JPY1M 0.36 %


<i> (b) </i> CHF most consistently exhibits a positive risk premium, followed by EUR. GBP and JPY have the most frequent negative risk premium.

<i> (c) </i> One could use expected relative risk-free rate variations to adjust positions based on FX risk premium for upcoming periods. Specifically, when risk-premium is predicted to be positive, one should favor increasing long and reducing short positions in the foreign currency. The opposite would be true for periods when risk-premium is predicted to be negative.